# Preprocessing and Feature Selection

In [1]:
from enum import IntEnum
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from pathlib import Path
import sys
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

from config import Config

data_path = Config.FARS_CLEANED_DIR / "fars.pkl"
df = pd.read_pickle(data_path)
df.head()

,ST_CASE,STATENAME,PEDS,PERNOTMVIT,VE_TOTAL,VE_FORMS,PVH_INVL,PERSONS,PERMVIT,COUNTYNAME,...,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,FATALS,TOTAL_HARM,INTOXICATED_DRIVER_INVOLVED,DRUNK_DRIVERS,DRIVERS_ON_DRUGS
0,10001,Alabama,0,0,2,1,1,1,1,TALLADEGA (121),...,NaN,6,15,88,88,1,4.0,False,0,0
1,10002,Alabama,0,0,1,1,0,2,2,WALKER (127),...,NaN,0,59,88,88,2,8.0,False,0,0
2,10003,Alabama,0,0,2,2,0,2,2,CHILTON (21),...,NaN,23,10,99,99,1,6.0,True,0,2
3,10004,Alabama,0,0,1,1,0,2,2,BALDWIN (3),...,NaN,13,14,88,88,1,4.0,False,0,0
4,10005,Alabama,0,0,2,2,0,2,2,JEFFERSON (73),...,NaN,7,28,88,88,1,6.0,False,0,0


### 2. Feature Engineering and Further Data Cleaning

We need to update/convert a few columns:
- ST_CASE is not needed anymore. 
- RAIL identifies if the crash occurred in or near a rail grade crossing. It is either 0000000 (not applicable), xxxxxxA (federal code for rail grade crossing) or 9999999 (unknown). Thus we will convert this to a binary feature. Treat 0000000 (N/A) and 9999999 (Unknown) as 0, everything else (actual codes) as 1
- Ensure severity metrics are numeric
- Convert time to cyclical encoding (model needs to see "night cycle"). For example, 23.00 is far from 01.00, but the number 23 is far from 1

In [3]:
# Rail Crossing Binary Feature
df['IS_RAIL_CROSSING'] = np.where(
    df['RAIL'].astype(str).isin(['0000000', '9999999', 'nan']), 
    0, 
    1
)

# Severity & On-Scene Metrics
severity_cols = ['FATALS', 'PERSONS', 'TOTAL_HARM']
for col in severity_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# Cyclical Time Encoding
df['HOUR_SIN'] = np.sin(2 * np.pi * df['HOUR'] / 24)
df['HOUR_COS'] = np.cos(2 * np.pi * df['HOUR'] / 24)

df.head()

,ST_CASE,STATENAME,PEDS,PERNOTMVIT,VE_TOTAL,VE_FORMS,PVH_INVL,PERSONS,PERMVIT,COUNTYNAME,...,HOSP_HR,HOSP_MN,FATALS,TOTAL_HARM,INTOXICATED_DRIVER_INVOLVED,DRUNK_DRIVERS,DRIVERS_ON_DRUGS,IS_RAIL_CROSSING,HOUR_SIN,HOUR_COS
0,10001,Alabama,0,0,2,1,1,1,1,TALLADEGA (121),...,88,88,1,4.0,False,0,0,0,1.000000,6.123234e-17
1,10002,Alabama,0,0,1,1,0,2,2,WALKER (127),...,88,88,2,8.0,False,0,0,0,0.000000,1.000000e+00
2,10003,Alabama,0,0,2,2,0,2,2,CHILTON (21),...,99,99,1,6.0,True,0,2,0,-0.500000,8.660254e-01
3,10004,Alabama,0,0,1,1,0,2,2,BALDWIN (3),...,88,88,1,4.0,False,0,0,0,-0.258819,-9.659258e-01
4,10005,Alabama,0,0,2,2,0,2,2,JEFFERSON (73),...,88,88,1,6.0,False,0,0,0,0.965926,-2.588190e-01


#### Save the feature-engineered dataset
This will be used fot the actual model training.

In [4]:
from config import Config
output_path = Config.FARS_CLEANED_DIR / "fars_model_ready.pkl"

df = df.convert_dtypes()
df.to_pickle(output_path)
print(f"Saved preprocessed data to {output_path}")

Saved preprocessed data to a:\Documents\Aarthi\Uni\Lisbon\Y1\Foundations of Data Science\Project\US_accidents_project\data\processed\fars\fars_model_ready.pkl


Now we need to select the features. We define categorical (ex. STATENAME, ROUTENAME,...) and numerical (ex. FATALS, IS_RAIL_CROSSING, ...) features separately to ensure easier processing. Moreover, since we will use a Random Forest to give features an importance score, we have to encode the high cardiandlty features with target encoding, since RF is very sensitiv to this.

In [5]:
target_enc_cols = ['STATENAME', 'ROUTENAME', 'HARM_EVNAME', 'REL_ROADNAME']
target = 'INTOXICATED_DRIVER_INVOLVED'

label_enc_cols = ['FUNC_SYSNAME', 'RUR_URBNAME', 'MONTH', 'DAY_WEEKNAME', 
                  'MAN_COLLNAME', 'LGT_CONDNAME', 'WEATHERNAME']

num_features = [
    'IS_RAIL_CROSSING', 'FATALS', 'PERSONS', 'TOTAL_HARM', 
    'HOUR_SIN', 'HOUR_COS' # Using cyclic hours instead of raw 'HOUR'
]

# Create X and y for feature selection
X = df[target_enc_cols + label_enc_cols + num_features].copy()
y = df[target]

We must split NOW, before encoding, to prevent leakage

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Target Encoding
We calculate the mean of the target (Impairment) for each category ONLY in the Training set. Then we map those values to the Test set.

In [7]:
global_mean = y_train.mean() # Fallback for new categories seen in test but not train

for col in target_enc_cols:
    # 1. Calculate average impairment per category (e.g., per State) in TRAIN
    # We combine X_train and y_train temporarily to group them
    train_temp = X_train.copy()
    train_temp['target'] = y_train
    
    # Calculate mappings
    category_means = train_temp.groupby(col)['target'].mean()
    
    # 2. Map these means to the columns
    # Create new column name (e.g., STATENAME_risk)
    new_col = col + '_risk_score'
    
    X_train[new_col] = X_train[col].map(category_means)
    X_test[new_col] = X_test[col].map(category_means)

    
    # 3. Handle Missing Values
    # If a category in Test wasn't in Train, fill with the global average
    # X_train[new_col] = X_train[new_col].fillna(global_mean)
    # X_test[new_col] = X_test[new_col].fillna(global_mean)

    #Temp fix because global mean gave an issue with categoricals
    X_train[new_col] = X_train[new_col].ffill()
    X_test[new_col] = X_test[new_col].ffill()
    
    # 4. Drop the original text column
    X_train = X_train.drop(columns=[col])
    X_test = X_test.drop(columns=[col])

C:\Users\aarth\AppData\Local\Temp\ipykernel_19028\3627461361.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  category_means = train_temp.groupby(col)['target'].mean()
C:\Users\aarth\AppData\Local\Temp\ipykernel_19028\3627461361.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  category_means = train_temp.groupby(col)['target'].mean()


#### Label Encoding
For the categorical features, we have to use an encoding. For this we will use the LabelEncoder from sklearn. For the numerical features, we will impute missing values with the median, if there still are any.

In [8]:
# Standard encoding for low-cardinality columns
le = LabelEncoder()
for col in label_enc_cols:
    # Fit on train, transform both (handling potential new labels with error bypass or string casting)
    # Using astype(str) ensures uniformity
    le.fit(X_train[col].astype(str))
    X_train[col] = le.transform(X_train[col].astype(str))
    
    # For test, we must handle unseen labels carefully. 
    # A quick hack for this demo is using fit_transform on test separately or ignoring errors,
    # but strictly, we should map unseen to a value. 
    # Here we re-fit on combined data just for the encoder structure to prevent crashing:
    combined_data = pd.concat([X_train[col], X_test[col]], axis=0).astype(str)
    le.fit(combined_data)
    X_test[col] = le.transform(X_test[col].astype(str))

Fill missing values of numeric features with the median.

In [9]:
# Fill numeric NaNs
# X_train = X_train.fillna(X_train.median())
# X_test = X_test.fillna(X_train.median())
X_train = X_train.ffill()
X_test = X_test.ffill()

X_train.head()

,FUNC_SYSNAME,RUR_URBNAME,MONTH,DAY_WEEKNAME,MAN_COLLNAME,LGT_CONDNAME,WEATHERNAME,IS_RAIL_CROSSING,FATALS,PERSONS,TOTAL_HARM,HOUR_SIN,HOUR_COS,STATENAME_risk_score,ROUTENAME_risk_score,HARM_EVNAME_risk_score,REL_ROADNAME_risk_score
4563,6,4,1,0,2,1,2,0,3,4,12,0.258819,0.965926,0.323829,0.323731,0.382811,0.290192
6342,7,4,2,2,0,0,2,0,1,3,10,0.5,0.866025,0.451868,0.337576,0.382811,0.290192
24048,0,4,2,5,2,0,2,0,1,3,4,-0.5,0.866025,0.453988,0.369305,0.382811,0.290192
31878,3,4,6,1,11,4,2,0,1,1,4,0.965926,-0.258819,0.333492,0.323731,0.091641,0.290192
17349,6,4,3,2,11,0,2,0,1,1,4,-0.965926,0.258819,0.326877,0.371397,0.091641,0.290192


#### Train & Evaluate
To see what features are imporant, we run a Random Forest:

In [10]:
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

# Feature Importance
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(importances)

                    Feature  Importance
13     STATENAME_risk_score    0.138266
15   HARM_EVNAME_risk_score    0.111622
2                     MONTH    0.103097
3              DAY_WEEKNAME    0.080506
12                 HOUR_COS    0.068508
9                   PERSONS    0.065128
11                 HOUR_SIN    0.065075
0              FUNC_SYSNAME    0.063353
14     ROUTENAME_risk_score    0.059023
10               TOTAL_HARM    0.054116
16  REL_ROADNAME_risk_score    0.041746
6               WEATHERNAME    0.041018
4              MAN_COLLNAME    0.040663
5              LGT_CONDNAME    0.034937
1               RUR_URBNAME    0.023344
8                    FATALS    0.008895
7          IS_RAIL_CROSSING    0.000704
